In [1]:
# 라이브러리 가져오기
import pandas as pd # Series, DataFrame 사용
import numpy as np # 값을 생성하는데 사용

### 계층 색인

- 행, 열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차원 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack() : 컬럼을 로우로 피벗
    - unstack() : 로우를 컬럼으로 피벗

In [60]:
# Series
# 인덱스에 다차원 리스트(아이템 2개)를 전달
# 다차원리스트[0] 상위계층
# 상위계층 작성시 주의점 : 각 계층별로 속하는 하위계층 값의 개수만큼 계층명 작성
# 상위계층 리스트 개수 = 하위계층 리스트 개수
# 다차원리스트[1] : 하위계층
# 실습 --->
# 로우 인덱스 - 상위계층 : a, b, c, d
# 하위계층 : a(1, 2, 3), b(1, 2), c(1, 2, 3, 4), d(1)
s1 = pd.Series(np.arange(10), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'd'],
                                   [1, 2, 3, 1, 2, 1, 2, 3, 4, 1]])
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [61]:
# 인덱스 확인
s1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('c', 4),
            ('d', 1)],
           )

In [62]:
# 상위계층에 접근
# 계층 색인이 적용된 객체에 상위 인덱스에 접근 : 일반적인 Series 인덱싱으로 접근
s1['c']

1    5
2    6
3    7
4    8
dtype: int32

In [63]:
# 계층색인에 대하 ㄴ슬라이싱 : 마지막 인덱스도 포함('b': 'd'로 입력시 b, c, d로 조회)
s1['b':'d']

b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [64]:
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [65]:
# 하위계층에 접근
# a, b, c, d에서 하위계층 라벨번호 2인 데이터만 조회
# 데이터[상위계층명, 하위계층번호]
s1[:, 2]

a    1
b    4
c    6
dtype: int32

In [66]:
# 하위 계층에 슬라이싱 : loc메서드를 이용해서 슬라이싱
# 상위계층이 c이고 하위계층을 2~4까지 슬라이싱(.loc는 마지막 번호도 포함)
s1['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

In [67]:
# unstack() : 최하위(기본동작) 에 있는 로우 계층을 컬럼으로 적용하여 위로 올림
# Series 객체를 DataFrame화 할 수 있음
# NaN : 기존에 없던 로우 계층에 대한 값
s1.unstack()

,1,2,3,4
a,0.0,1.0,2.0,NaN
b,3.0,4.0,NaN,NaN
c,5.0,6.0,7.0,8.0
d,9.0,NaN,NaN,NaN


In [68]:
# stack() 메서드 : 컬럼에 있던 값을 가장 하위 인덱스로 내려서 재배열
# DataFrame을 다시 Series화 할 때 사용
s1.unstack().stack()

a  1    0.0
   2    1.0
   3    2.0
b  1    3.0
   2    4.0
c  1    5.0
   2    6.0
   3    7.0
   4    8.0
d  1    9.0
dtype: float64

In [69]:
# DataFrame 생성
# 구조 : 4 * 5
# 로우 인덱스 : 상위(2017, 2018) / 하위(모든 상위 인덱스에 대해 동일하게 a, b)
# 컬럼 인덱스 : 상위(서울, 경기) / 하위(서울-강남, 잠실 / 경기-안양, 수원, 판교)
# 값 : 1씩 증가하는 20개
df = pd.DataFrame(np.arange(20).reshape(4, 5),
                 index=[[2017, 2017, 2018, 2018],
                       ['a', 'b', 'a', 'b']],
                 columns=[['서울', '서울', '경기', '경기', '경기'],
                         ['강남', '잠실', '안양', '수원', '판교']])
df

서울      경기        
        강남  잠실  안양  수원  판교
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

In [70]:
# 컬럼의 상위 계층 접근 : 일반적인 df컬럼 인덱싱 방식
# '서울' 데이터 조회
df['서울']

강남  잠실
2017 a   0   1
     b   5   6
2018 a  10  11
     b  15  16

In [71]:
# 컬럼이 이중으로 구조가 만들어져있으므로 인덱싱도 이중으로 해야 하나의 요소를 조회할 수 있음
df['서울']['강남']

2017  a     0
      b     5
2018  a    10
      b    15
Name: 강남, dtype: int32

In [72]:
# 튜플로 전달시 하나의 리스트로 2개 이상의 계층 처리 가능
df[('서울', '강남')]

2017  a     0
      b     5
2018  a    10
      b    15
Name: (서울, 강남), dtype: int32

In [73]:
# row의 상위 계층 접근
# 2017년 데이터를 조회해주세요.
df.loc[2017]

서울    경기      
  강남 잠실 안양 수원 판교
a  0  1  2  3  4
b  5  6  7  8  9

In [74]:
# 2017년도 전반기(a) 데이터 조회
df.loc[2017].loc['a']

서울  강남    0
    잠실    1
경기  안양    2
    수원    3
    판교    4
Name: a, dtype: int32

In [75]:
df.loc[(2017, 'a')]

서울  강남    0
    잠실    1
경기  안양    2
    수원    3
    판교    4
Name: (2017, a), dtype: int32

In [76]:
# 분당 ~ 수원까지 데이터 조회
# 주의점 : 데이터프레임 컬럼은 row 슬라이싱 결과에 대해서만 슬라이싱 적용 가능
# df['경기']['안양':'수원'] -> error
df['경기'].loc[:, "안양":"수원"]

안양  수원
2017 a   2   3
     b   7   8
2018 a  12  13
     b  17  18

In [77]:
# row의 최하위계층(a, b)를 컬럼으로 올리기 : unstack()
df.unstack()

서울              경기                    
      강남      잠실      안양      수원      판교    
       a   b   a   b   a   b   a   b   a   b
2017   0   5   1   6   2   7   3   8   4   9
2018  10  15  11  16  12  17  13  18  14  19

In [78]:
# 컬럼 최하위계층을 로우의 하위계층으로 재배열 : stack()
df.stack()

경기    서울
2017 a 강남   NaN   0.0
       수원   3.0   NaN
       안양   2.0   NaN
       잠실   NaN   1.0
       판교   4.0   NaN
     b 강남   NaN   5.0
       수원   8.0   NaN
       안양   7.0   NaN
       잠실   NaN   6.0
       판교   9.0   NaN
2018 a 강남   NaN  10.0
       수원  13.0   NaN
       안양  12.0   NaN
       잠실   NaN  11.0
       판교  14.0   NaN
     b 강남   NaN  15.0
       수원  18.0   NaN
       안양  17.0   NaN
       잠실   NaN  16.0
       판교  19.0   NaN

In [79]:
# 계층의 인덱스번호 또는 라벨을 사용하여 상, 하위간 교환
# swaplevel(key1, key2, axis=0(기본값))
# axis가 0인 경우는 row의 상하위 개념이 뒤집힌다.
df.swaplevel(1, 0)

서울      경기        
        강남  잠실  안양  수원  판교
a 2017   0   1   2   3   4
b 2017   5   6   7   8   9
a 2018  10  11  12  13  14
b 2018  15  16  17  18  19

In [80]:
# swaplevel에 axis=1을 주면 계층이 아닌 컬럼에 변동이 생깁니다.
df.swaplevel(1, 0, axis=1)

강남  잠실  안양  수원  판교
        서울  서울  경기  경기  경기
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

### 객체 복사

- 할당기호(=) / 슬라이싱 : 원본과 상호 종속적인 복사본 객체 생성(얕은복사)
- obj.copy() : 원본과 독립적인 복사본 객체 생성(깊은복사)

In [81]:
# 모든 값이 1인 5 * 1 구조의 Series 생성
s1 = pd.Series(np.ones(5))
s1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [82]:
# 할당기호 = 로 복사 : 얕은복사
s2 = s1

In [83]:
# 할당기호로 복사한 객체의 아이템 수정
s2[0] = 10
s2

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [84]:
# = 을 이용한 대입은 원본객체도 변경됨
s1

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [85]:
# 원본객체의 아이템 수정
s1[1] = 3
s1

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [86]:
s2

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [87]:
# 슬라이싱으로 복사(일반 파이썬 -> 깊은복사, pandas->얕은복사로 취급됨)
s3 = s1[:]
s3

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [88]:
# 복사본 객체 아이템 수정
s3[2] = -9
s3

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64

In [89]:
# s1과 s2 확인
print(s1)
print("------")
print(s2)

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64
------
0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64


In [90]:
# 원본 변경
s1[3] = 33
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [91]:
# 복사본 객체로 적용받음
print(s2)
print(s3)

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64
0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64


In [92]:
# .copy()를 이용한 깊은복사(복사)
s4 = s1.copy()

In [93]:
# s4 수정(s1 영향 X)
s4[4] = 44

In [94]:
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

In [95]:
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [96]:
# s1을 변경하는 경우에도 s4는 영향받지않음
s1[0] = 0

In [97]:
s1

0     0.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [98]:
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

### 정렬

- obj.sort_index() : 인덱스를 기준으로 정렬 (기본값은 ascending=True, 오름차순 정렬)
    - DataFrame, Series
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis = 1 : 컬럼 인덱스 기준으로 정렬
- obj.sort_values() : 값을 기준으로 정렬
    - DataFrame, Series
        - by : 정렬의 기준이 되는 인덱스 값 전달
        - axis = 0 : 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis = 1 : 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 또는 라벨명 전달

In [99]:
# Series생성
# 값과 인덱스라벨이 순서대로 들어가지 않은 Series
s1 = pd.Series([2, 3, 1, 7, 0], index=list('gacfd'))
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [100]:
# 인덱스 기준 오름차순으로 정렬
# 기본동작 : 오름차순 & 로우 인덱스
s1.sort_index()

a    3
c    1
d    0
f    7
g    2
dtype: int64

In [101]:
# 인덱스 기준 내림차순으로 정렬
s1.sort_index(ascending=False)

g    2
f    7
d    0
c    1
a    3
dtype: int64

In [102]:
# 일시적 변경이므로 s1에는 영향을 주지 않음
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [103]:
# 값 기준 오름차순으로 정렬
s1.sort_values()

d    0
c    1
g    2
a    3
f    7
dtype: int64

In [104]:
# 값 기준 내림차순으로 정렬
s1.sort_values(ascending=False)

f    7
a    3
g    2
c    1
d    0
dtype: int64

In [105]:
# 원본에 바로 반영되지 않음
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [106]:
# DataFrame 생성
# 4 * 5, 무작위 정수(np.random.randint())
# 로우/컬럼 인덱스도 순서가 없는 값 지정
# 로우 'hcae', 컬럼은 'EAFCD'
df1 = pd.DataFrame(np.random.randint(20, size=(4, 5)), index=list('hcae'), columns=list('EAFCD'))
df1

,E,A,F,C,D
h,18,12,4,9,11
c,9,16,10,4,17
a,18,12,3,13,15
e,16,15,5,2,17


In [107]:
# 로우 인덱스 기준으로 오름차순 정렬(axis=0)
df1.sort_index()

,E,A,F,C,D
a,18,12,3,13,15
c,9,16,10,4,17
e,16,15,5,2,17
h,18,12,4,9,11


In [108]:
# 로우 인덱스 기준으로 내림차순 정렬(axis=0)
df1.sort_index(ascending=False)

,E,A,F,C,D
h,18,12,4,9,11
e,16,15,5,2,17
c,9,16,10,4,17
a,18,12,3,13,15


In [109]:
# 컬럼명 기준으로 오름차순 정렬(axis=1)
df1.sort_index(axis=1)

,A,C,D,E,F
h,12,9,11,18,4
c,16,4,17,9,10
a,12,13,15,18,3
e,15,2,17,16,5


In [110]:
# 컬럼명 기준으로 내림차순 정렬
df1.sort_index(axis=1, ascending=False)

,F,E,D,C,A
h,4,18,11,9,12
c,10,9,17,4,16
a,3,18,15,13,12
e,5,16,17,2,15


In [111]:
# 컬럼 기준으로 내림차순 정렬 후, 로우 기준으로 오름차순 정렬
df1.sort_index(axis=1, ascending=False).sort_index()

,F,E,D,C,A
a,3,18,15,13,12
c,10,9,17,4,16
e,5,16,17,2,15
h,4,18,11,9,12


In [112]:
# 원본에는 변경 없음
df1

,E,A,F,C,D
h,18,12,4,9,11
c,9,16,10,4,17
a,18,12,3,13,15
e,16,15,5,2,17


In [113]:
# 값 기준으로 정렬
# 컬럼 D의 값을 기준으로 오름차순 정렬
# sort_values(axis=0, by='기준컬럼명') : 기본동작(정렬결과를 로우에 반영)
df1.sort_values(by='D')

,E,A,F,C,D
h,18,12,4,9,11
a,18,12,3,13,15
c,9,16,10,4,17
e,16,15,5,2,17


In [114]:
# 컬럼 A를 기준으로 내림차순 정렬
df1.sort_values(by='A', ascending=False)

,E,A,F,C,D
c,9,16,10,4,17
e,16,15,5,2,17
h,18,12,4,9,11
a,18,12,3,13,15


In [115]:
# 인덱스라벨(row) c의 값을 오름차순으로 정렬
# 결과적으로 정렬되는 대상 : 컬럼
# 정렬의 기준 : row 라벨
df1.sort_values(axis=1, by='c')

,C,E,F,A,D
h,9,18,4,12,11
c,4,9,10,16,17
a,13,18,3,12,15
e,2,16,5,15,17


In [116]:
# 로우 'e'를 기준으로 값을 내림차순으로 정렬
df1.sort_values(axis=1, by='e', ascending=False)

,D,E,A,F,C
h,11,18,12,4,9
c,17,9,16,10,4
a,15,18,12,3,13
e,17,16,15,5,2


In [119]:
# 두 개의 컬럼에 대해서 정렬 : 리스트로 묶어서 by의 인자값으로 전달
# 정렬 우선순위 : 차례대로 1순위 > 2순위(1순위 동점시 2순위로 넘어감)
# 1순위 정렬 후 1순위 컬럼의 동점값에 대해 2순위 컬럼의 값이 재정렬
# 동일한 값이 아니라면 무조건 1순위 컬럼이 우선
df1.sort_values(by=['D', 'A'])

,E,A,F,C,D
h,18,12,4,9,11
a,18,12,3,13,15
e,16,15,5,2,17
c,9,16,10,4,17


In [120]:
# D는 오름차순 A는 내림차순으로 하고 싶다면
# ascending = [True, False] 와 같이 각각의 컬럼에 대해서 ascending을 따로 매겨준다
df1.sort_values(by=['D', 'A'], ascending=[True, False])

,E,A,F,C,D
h,18,12,4,9,11
a,18,12,3,13,15
c,9,16,10,4,17
e,16,15,5,2,17


### 연습문제

#### 1. 아래의 데이터프레임을 생성하세요

- 학생들의 점수는 50 이상 100 미만의 무작위 정수 값을 생성하여 사용

<img src="img/df_sort_practice1.png" width=250 align='left'>

In [149]:
# 데이터프레임 생성
df = pd.DataFrame(np.random.randint(50, 100, 20).reshape(5, 4), 
             index=['Kim', 'Park', 'Lee', 'Jung', 'Moon'],
            columns=[[2016, 2016, 2017, 2017],['영어', '수학', '영어', '수학']])
df

2016     2017    
       영어  수학   영어  수학
Kim    86  60   97  88
Park   74  76   67  53
Lee    73  89   77  69
Jung   73  55   99  88
Moon   52  99   72  62

In [150]:
# 컬럼 인덱스 이름 설정(리스트로 넘김)
#멀티 인덱스는 무조건 set_names를 이용해야 입력 가능
#df.columns.name=['연도', '과목명']
df.columns.set_names(['연도', '과목명'], inplace=True)

In [151]:
df

연도   2016     2017    
과목명    영어  수학   영어  수학
Kim    86  60   97  88
Park   74  76   67  53
Lee    73  89   77  69
Jung   73  55   99  88
Moon   52  99   72  62

In [152]:
# 로우 인덱스 '학생명' 으로 설정
df.index.name = '학생명'

In [153]:
df

연도   2016     2017    
과목명    영어  수학   영어  수학
학생명                   
Kim    86  60   97  88
Park   74  76   67  53
Lee    73  89   77  69
Jung   73  55   99  88
Moon   52  99   72  62

#### 2. 2016년 데이터만 별도의 데이터프레임으로 분리 저장하세요.(깊은복사)

In [154]:
df2016 = df[2016].copy()
df2016

과목명,영어,수학
학생명,,
Kim,86,60
Park,74,76
Lee,73,89
Jung,73,55
Moon,52,99


#### 3. 2016년 데이터에 대해 학생 이름을 기준으로 오름차순 정렬하세요.

In [155]:
df2016.sort_index()

과목명,영어,수학
학생명,,
Jung,73,55
Kim,86,60
Lee,73,89
Moon,52,99
Park,74,76


#### 4. 2016년 데이터의 과목별 점수에 대하여 영어 점수가 높은 순서로 정렬하고, 영어 점수 동점자의 경우 수학 점수가 낮은 순서로 정렬하세요.

In [158]:
df2016.sort_values(by=['영어', '수학'], ascending=[False, True])

과목명,영어,수학
학생명,,
Kim,86,60
Park,74,76
Jung,73,55
Lee,73,89
Moon,52,99


### 데이터 합치기

- merge
    - pandas 객체의 메서드로 pandas.merge(df1, df2 ...)로 사용
    - 두 개의 데이터프레임에 대해 특정 컬럼을 기준으로 합치기
    - 주요 파라미터
        - how : 합치는 방식으로 inner(기본값), left, right, outer 방식 존재
        - on : 합치는 기준으로 두 개의 데이터프레임에 공통으로 존재하는 컬럼명을 사용해야함 (기본값=None)

In [159]:
# 고객번호와 고객명을 저장하고 있는 데이터프레임 생성
# 딕셔너리 타입으로 데이터프레임 데이터 생성 : key:col, value-> list() : row
df1 = pd.DataFrame({'no':[30, 31, 32, 33, 34],
                  'name':['김파이썬', '이장고', '박팬더스', '강넘파이', '최주피터']})
df1

,no,name
0,30,김파이썬
1,31,이장고
2,32,박팬더스
3,33,강넘파이
4,34,최주피터


In [160]:
# 고객 번호와 주문수량을 저장하고 있는 데이터프레임 생성
# 다른 고객번호(no)를 일부 지정
df2 = pd.DataFrame({'no':[30, 33, 40, 41],
                   'amount':[100, 130, 40, 60]})
df2

,no,amount
0,30,100
1,33,130
2,40,40
3,41,60


In [161]:
# 가장 기본적인 merge 방식 : 공통 컬럼을 기준으로 두 df에 모두 존재하는 교집합만 추출
# 기본 동작 : how = 'inner'
# df1 & df2의 공통컬럼 : no
# 공통 컬럼 내부의 공통 값 : 30, 33
# df1 : 30 ~ 34
# df2 : 30, 33, 40, 41
pd.merge(df1, df2)

,no,name,amount
0,30,김파이썬,100
1,33,강넘파이,130


In [162]:
# how='outer' : 결합 기준으로 공통 컬럼을 사용함
# 교집합이 아닌 부분에 대해서는 NaN으로 처리(누락되는 데이터 없음)
# 합집합 형태로 만들어줌
pd.merge(df1, df2, how="outer")

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,NaN
3,33,강넘파이,130.0
4,34,최주피터,NaN
5,40,NaN,40.0
6,41,NaN,60.0


In [163]:
# left : 첫 번째로 전달한 df1의 데이터는 모두 살리고, df2에서는 df1과 겹칠때만 남김
pd.merge(df1, df2, how='left')

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,NaN
3,33,강넘파이,130.0
4,34,최주피터,NaN


In [164]:
# right : 두 번째 데이터프레임의 데이터를 보존
# df1에서는 df2와 겹치는 경우에 한해 데이터를 남김
pd.merge(df1, df2, how='right')

,no,name,amount
0,30,김파이썬,100
1,33,강넘파이,130
2,40,NaN,40
3,41,NaN,60


In [165]:
# 공통 컬럼이 두 개 이상인 경우
# '고객명', '날짜', '정보' 컬럼 3개에 데이터를 저장하고 있는 데이터프레임 생성
df3 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                   '날짜':['2020-06-01', '2020-06-10', '2020-06-14'],
                   '정보':['010', '011', '019']})
df3

,고객명,날짜,정보
0,김파이썬,2020-06-01,010
1,이장고,2020-06-10,011
2,박팬더스,2020-06-14,019


In [166]:
# 고객명, 정보를 저장하고 있는 데이터프레임 생성
df4 = pd.DataFrame({'고객명':['김파이썬', '박팬더스', '최넘파이'],
                   '정보':['F', 'M', 'M']})
df4

,고객명,정보
0,김파이썬,F
1,박팬더스,M
2,최넘파이,M


In [167]:
# 공통된 이름을 가진 컬럼이 2개 이상인 경우, 파라미터가 없으면 결과도 없다
pd.merge(df3, df4)

,고객명,날짜,정보


In [168]:
# on 파라미터 : 공통된 컬럼이 여럿인 경우 결합 기준 컬럼을 지정
# 합칠때 key로 사용할 컬럼 : '고객명'
# 공통컬럼 결과 : 고객명, 정보 -> 정보 컬럼 결과를 확인
# 기본동작으로 merge : inner
pd.merge(df3, df4, on='고객명')

,고객명,날짜,정보_x,정보_y
0,김파이썬,2020-06-01,010,F
1,박팬더스,2020-06-14,019,M


In [169]:
# left_on, right_on : 두 개의 데이터프레임에 대해서 서로 다른 기준컬럼을 지정
# 예) 동일한 속성의 자료를 저장하는 컬럼인데 이름이 다른 경우
# 고객이름, 날짜, 구매금액을 저장하고 있는 데이터프레임 생성
df5 = pd.DataFrame({"고객이름":['김파이썬', '박팬더스', '강주피터'],
                   "날짜":['2020-01-01', '2020-02-01', '2020-02-15'],
                   '구매금액':[1, 2, 3]})
df5

,고객이름,날짜,구매금액
0,김파이썬,2020-01-01,1
1,박팬더스,2020-02-01,2
2,강주피터,2020-02-15,3


In [170]:
# 고객명, 성별을 지정하고 있는 데이터프레임을 생성
df6 = pd.DataFrame({'고객명':['김파이썬', '박팬더스'],
                   '성별':['F', 'M']})
df6

,고객명,성별
0,김파이썬,F
1,박팬더스,M


In [171]:
# 겹치는 컬럼명이 없어서 에러 발생
pd.merge(df5, df6)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [172]:
# left(df5), right(df6)에서 공통컬럼명을 각각 지정
tmp = pd.merge(df5, df6, left_on='고객이름', right_on='고객명')
tmp

,고객이름,날짜,구매금액,고객명,성별
0,김파이썬,2020-01-01,1,김파이썬,F
1,박팬더스,2020-02-01,2,박팬더스,M


In [178]:
# drop 메서드를 이용해 '고객명' 컬럼을 삭제해주세요
#tmp.drop('고객명', axis=1, inplace=True)
tmp.drop(columns='고객명', inplace=True)
tmp

,고객이름,날짜,구매금액,성별
0,김파이썬,2020-01-01,1,F
1,박팬더스,2020-02-01,2,M


#### concat

- 특정 key를 기준으로 데이터를 합치는 것이 아니라 행, 열 기준으로 데이터를 연결
- 주요 파라미터
    - axis : 0 / 행 방향(기본값)이며 컬럼을 key로 합치고, 1 / 열 방향으로 로우를 key로 합침
    - join : 데이터프레임끼리 연결할 때 합치는 방법으로 outer(기본값), inner 방식 존재
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [179]:
# 공통 인덱스 라벨을 가지는 Series 2개 생성
s1 = pd.Series([1, 2, 3], index=list('abc'))
s2 = pd.Series([5, 6, 7, 8], index=list('abfh'))
print(s1)
print(s2)

a    1
b    2
c    3
dtype: int64
a    5
b    6
f    7
h    8
dtype: int64


In [180]:
# 두 Series간 연결
# 기본 : axis=0(행 방향으로 연결)
# 첫 번째로 전달된 객체가 위에, 두 번째로 전달된 객체가 아래로 추가(연결)
# 인덱스 라벨은 기존 값 유지
pd.concat([s1, s2])

a    1
b    2
c    3
a    5
b    6
f    7
h    8
dtype: int64

In [181]:
# 새로운 인덱스로 초기화하기
pd.concat([s1, s2], ignore_index=True)

0    1
1    2
2    3
3    5
4    6
5    7
6    8
dtype: int64

In [182]:
# 열 방향  연결 -> 두 개의 Series를 연결해서 하나의 DataFrame으로 만들기
# 길이가 다른 경우
pd.concat([s1, s2], axis=1)

,0,1
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [186]:
# 열 이름을 설정하면서 붙여주기 : keys 파라미터에 컬럼명을 리스트로 전달
pd.concat([s1, s2], axis=1, keys=['c1', 'c2'])

,c1,c2
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [188]:
# 두 개의 데이터프레임 연결
# 고객명, 날짜, 금액
df1 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                  '날짜':['2020-06-01', '2020-06-02', '2020-06-14'],
                  '구매금액':[1, 2, 3]})
# 고객명, 성별
df2 = pd.DataFrame({'고객명':['김파이썬', '최넘파이'],
                   '성별':['F', 'M']})

,고객명,성별
0,김파이썬,F
1,최넘파이,M


In [189]:
# 행 방향으로(row개수 증가) 데이터 프레임 연결
pd.concat([df1, df2])

,고객명,날짜,구매금액,성별
0,김파이썬,2020-06-01,1.0,NaN
1,이장고,2020-06-02,2.0,NaN
2,박팬더스,2020-06-14,3.0,NaN
0,김파이썬,NaN,NaN,F
1,최넘파이,NaN,NaN,M


In [190]:
# 열 방향으로 데이터프레임 연결
pd.concat([df1, df2], axis=1)

,고객명,날짜,구매금액,고객명,성별
0,김파이썬,2020-06-01,1,김파이썬,F
1,이장고,2020-06-02,2,최넘파이,M
2,박팬더스,2020-06-14,3,NaN,NaN


In [191]:
df3 = pd.DataFrame({'지역':['서울', '부산', '제주'],
                   '직업':['웹개발자', '회계사', '회사원']})
df3

,지역,직업
0,서울,웹개발자
1,부산,회계사
2,제주,회사원


In [192]:
# 전제조건 : df1과 df3의 데이터가 순서, 맥락상 맞는것이 확정적일때 concat으로 합치면 좋스빈다.
pd.concat([df1, df3], axis=1)

,고객명,날짜,구매금액,지역,직업
0,김파이썬,2020-06-01,1,서울,웹개발자
1,이장고,2020-06-02,2,부산,회계사
2,박팬더스,2020-06-14,3,제주,회사원


### append
- 행 방향으로 데이터를 연결
- 주요 파라미터
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [193]:
# 두 개의 시리즈 생성
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([4, 5, 6, 7])

In [194]:
# 행 방향으로 데이터 연결하고 기존 인덱스 유지
s1.append(s2)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_15700\3431566202.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s1.append(s2)


0    1
1    2
2    3
0    4
1    5
2    6
3    7
dtype: int64

In [195]:
# 행 방향으로 데이터 연결하고 기존 인덱스 누락시키기
s1.append(s2, ignore_index=True)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_15700\4246903571.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s1.append(s2, ignore_index=True)


0    1
1    2
2    3
3    4
4    5
5    6
6    7
dtype: int64

In [196]:
# 두 개의 데이터프레임
# 서로 다른 컬럼명
df1 = pd.DataFrame([[1, 2], 
                    [3, 4]], columns=list('ab'))
df2 = pd.DataFrame([[5, 6], 
                    [7, 8], 
                    [9, 10]], columns=list('AB'))
print(df1)
print(df2)

   a  b
0  1  2
1  3  4
   A   B
0  5   6
1  7   8
2  9  10


In [197]:
df1.append(df2)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_15700\3062608662.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2)


,a,b,A,B
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
0,NaN,NaN,5.0,6.0
1,NaN,NaN,7.0,8.0
2,NaN,NaN,9.0,10.0


In [198]:
# 두 개의 데이터프레임
# 동일한 컬럼명
df3 = pd.DataFrame([[10, 20],
                   [30, 40],
                   [50, 60]], columns=list('ab'))
df3

,a,b
0,10,20
1,30,40
2,50,60


In [199]:
# 동일한 컬럼명에 대해서 append를 하면 컬럼갯수는 원본 데이터 숫자 그대로, 행만 추가
# append 는 보통 이런 케이스에 많이 사용함
df1.append(df3)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_15700\364716140.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df3)


,a,b
0,1,2
1,3,4
0,10,20
1,30,40
2,50,60


In [200]:
# ignore_index를 걸면 인덱스번호도 0부터 다시 초기화
df1.append(df3, ignore_index=True)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_15700\1777171504.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df3, ignore_index=True)


,a,b
0,1,2
1,3,4
2,10,20
3,30,40
4,50,60


In [201]:
# 시리즈와 데이터프레임
# 데이터프레임에 한 줄 추가할때 보통 시리즈를 많이 사용함
# 시리즈 라벨 & 데이터프레임 컬럼명이 서로 일치할때 매칭됨
# 시리즈 라벨, 데이터프레임 컬럼명을 모두 a,b,c,d로 매칭시킴
s1 = pd.Series([10, 20, 30, 40], index=list('abcd'))
df1 = pd.DataFrame([[1, 1, 1, 1],
                   [2, 2, 2, 2]], columns=list('abcd'))

In [202]:
s1

a    10
b    20
c    30
d    40
dtype: int64

In [203]:
df1

,a,b,c,d
0,1,1,1,1
1,2,2,2,2


In [205]:
# Series의 라벨이 df의 컬럼과 매핑이 됨
# 이 때 Series는 df의 새로운 row로 들어갑니다.
df1.append(s1, ignore_index=True)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_15700\639640000.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(s1, ignore_index=True)


,a,b,c,d
0,1,1,1,1
1,2,2,2,2
2,10,20,30,40


### 집계

- groupby(컬럼명)
    - 특정 속성을 기준으로 묶어서 다양한 집계 함수 적용
    - 대표적인 집계 함수
        - sum : 총합
        - mean : 평균값
        - min : 최소값
        - max : 최대값
        - count : 개수
        - std : 표준편차
- pivot table
    - df.pivot(로우로 사용될 컬럼명, 컬럼으로 사용될 컬럼명, 튜플을 구성하는 값으로 사용될 컬럼명, 집계함수)
    - 일차원으로 컬럼 및 로우가 단순 나열된 형식은 데이터를 파악하는데 적합하지 않기 때문에 pivot을 통해 계층 색인 및 형태 변경을 수행

In [16]:
# 엑셀 데이터 적재
# Os error -> engine='python'
# Unicode, Encoding -> encoding='utf-8' or 'cp949' or 'utf-16'
data = pd.read_excel('data/인구수예제.xlsx')
data

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [17]:
# 데이터 구조 확인
data.shape

(50, 6)

In [18]:
# 상위 5개만 조회하기
data.head()

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190


In [19]:
# 하위 5개만 조회하기
data.tail()

,도시,자치구,연도,남자인구,여자인구,총인구
45,부산,동래구,2013,83,65,148
46,부산,동래구,2014,139,87,226
47,부산,동래구,2015,147,115,262
48,부산,동래구,2016,61,102,163
49,부산,동래구,2017,132,105,237


In [20]:
# 상위 10개 조회
data.head(10)

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [21]:
# 자치구별 남, 여 인구 각각의 총합
# 조사년도는 총 5개년
data.groupby('자치구')[['남자인구', '여자인구']].sum()

,남자인구,여자인구
자치구,,
강남구,539,513
도봉구,485,550
동래구,562,474
동작구,454,582
서대문구,488,543
송파구,415,559
수영구,502,559
영등포구,629,562
종로구,483,373


In [22]:
# 도시별 남여 인구의 총합을 구해주세요.
data.groupby('도시')[['여자인구', '남자인구']].sum()

,여자인구,남자인구
도시,,
부산,1548,1684
서울,3682,3493


In [23]:
# 연도별, 도시별 과 같이 2개이상의 조건을 묶고싶을때는 .groupby([1번그룹, 2번그룹...])
# 형식으로 묶을수도 있습니다.
# 연도별, 도시별 남여인구 총합을 출력해주세요.
data.groupby(['연도', '도시'])[['여자인구', '남자인구']].sum()

여자인구  남자인구
연도   도시            
2013 부산   262   341
     서울   769   758
2014 부산   369   314
     서울   691   882
2015 부산   266   331
     서울   710   541
2016 부산   376   276
     서울   722   671
2017 부산   275   422
     서울   790   641

In [24]:
# 도시별 연도별 총인구 평균을 구해주세요.
data.groupby(['도시', '연도'])['총인구'].mean()

도시  연도  
부산  2013    201.000000
    2014    227.666667
    2015    199.000000
    2016    217.333333
    2017    232.333333
서울  2013    218.142857
    2014    224.714286
    2015    178.714286
    2016    199.000000
    2017    204.428571
Name: 총인구, dtype: float64

In [25]:
# 가독성을 높이기 위해 unstack() 사용
data.groupby(['도시', '연도'])['총인구'].mean().unstack()

연도,2013,2014,2015,2016,2017
도시,,,,,
부산,201.000000,227.666667,199.000000,217.333333,232.333333
서울,218.142857,224.714286,178.714286,199.000000,204.428571


In [12]:
# 마지막으로 여러분들이 집계 아무거나 여태까지 안쓴것중 궁금한거 한 번 써서 구문을 저한테 보내주세요.

In [28]:
# 위의 가독성 높이기에서 좀 더 가독성을 높인 구조
# unstack()은 하위 멀티인덱스를 컬럼으로 올리는 역할을 기본적으로 수행합니다.
# 하지만 unstack() 과 같이 인덱스번호를 넣어주면 상위 인덱스를 컬럼으로 보낼수도 있습니다.
data.groupby(['도시', '연도'])['총인구'].mean().unstack(0)

도시,부산,서울
연도,,
2013,201.000000,218.142857
2014,227.666667,224.714286
2015,199.000000,178.714286
2016,217.333333,199.000000
2017,232.333333,204.428571


#### groupby 결과를 pivot_table로 실행할 수 있다.


In [33]:
# 연도별 & 도시별 & 남여인구 각각의 최대값
# pivot_table(index=로우로 사용할 컬럼명, columns=컬럼으로 사용할 컬럼명
#             values=집계함수를 적용할 값으로  사용할 컬럼, aggfunc=사용할 집계함수명)
# 로우 : 연도, 도시
# data.groupby(['연도', '도시'])[['남자', '여자']].sum()
data.pivot_table(index=['연도', '도시'], values=['남자인구', '여자인구'], aggfunc='sum')

남자인구  여자인구
연도   도시            
2013 부산   341   262
     서울   758   769
2014 부산   314   369
     서울   882   691
2015 부산   331   266
     서울   541   710
2016 부산   276   376
     서울   671   722
2017 부산   422   275
     서울   641   790

In [34]:
# .unique()는 출현한 요소를 한 번씩만 출력한다.
data.도시.unique()

array(['서울', '부산'], dtype=object)

In [39]:
# 도시별, 연도별 총인구 평균
# 로우 : 연도
# 값은 총인구
# 집계 : 평균
data.pivot_table(index=['연도'], columns=['도시'], values=['총인구'], aggfunc='mean')

총인구            
도시            부산          서울
연도                          
2013  201.000000  218.142857
2014  227.666667  224.714286
2015  199.000000  178.714286
2016  217.333333  199.000000
2017  232.333333  204.428571

In [48]:
# 여러개의 집계함수를 서로 다른값에 사용하기
# 도시별, 자치구별 남자인구의 평균, 여자인구의 합계
# 로우 : 도시, 자치구
# 값 : 남자인구, 여자인구
# 집계함수는 딕셔너리 형식으로 {'컬럼명1':'사용할함수1', '컬럼명2':'사용할함수2'...}
tmp = data.pivot_table(index=['도시', '자치구'], values=['남자인구', '여자인구'], aggfunc={'남자인구':'mean', '여자인구':'sum'})
tmp.columns=['남자인구_평균', '여자인구_총합']
tmp

남자인구_평균  여자인구_총합
도시 자치구                   
부산 동래구     112.4      474
   수영구     100.4      559
   해운대구    124.0      515
서울 강남구     107.8      513
   도봉구      97.0      550
   동작구      90.8      582
   서대문구     97.6      543
   송파구      83.0      559
   영등포구    125.8      562
   종로구      96.6      373

### pivot_table 실습

In [65]:
# 1. 도시별 자치구별 남자인구 평균(data 변수 활용하기)
dpt = data.pivot_table(index=['자치구'], values=['남자인구'], aggfunc='mean')
dpt

,남자인구
자치구,
강남구,107.8
도봉구,97.0
동래구,112.4
동작구,90.8
서대문구,97.6
송파구,83.0
수영구,100.4
영등포구,125.8
종로구,96.6


In [52]:
# 남자인구 컬럼을 남자인구_평균 컬럼으로 이름 변경하기
dpt.columns=['남자인구_평균']

In [53]:
# 바뀌었는지 상위 3개 행만 출력해 확인하기(head 활용)
dpt.head(3)

남자인구_평균
도시 자치구          
부산 동래구     112.4
   수영구     100.4
   해운대구    124.0

In [64]:
# 자치구별 평균 남자인구를 인구수 순서대로 5개 출력해주세요.
dpt.sort_values(by='남자인구_평균', ascending=False).head(5)

남자인구_평균
도시 자치구          
서울 영등포구    125.8
부산 해운대구    124.0
   동래구     112.4
서울 강남구     107.8
부산 수영구     100.4

### 통계

- 주요 통계 함수
    - value_counts() : 각 고유값의 중복개수
    - count : NaN 값을 제외한 값의 개수
    - describe : 각 컬럼에 대한 요약통계 계산(count, mean, std, min, 1사분위수, 중위값, 3사분위수, max)
    - min, max : 최소, 최대 값
    - sum : 총 합
    - cumsum : 누적합
    - mean : 평균
    - median : 중위값(전체 데이터를 나열 했을 때 중간에 위치한 값)
    - var : 분산(데이터가 전체적으로 흩어진 정도, 편차제곱의 평균)
    - std : 표준편차 = 분산의 양의 제곱근
- 주요 파라미터
    - axis : 연산의 기준이 되는 축, axis=0(기본값)이면 행 방향으로 axis=1이면 열 방향으로 적용
    - skipna : NaN 값을 제외할지 여부를 설정, 기본값 = True

In [73]:
# 샘플 데이터 생성, 50이상 100미만의 난수 데이터 20개를 5 * 4 형태로
# 로우명 : ['Kim', 'Park', 'Lee', 'Jung', 'Moon']
# 컬럼명 : 2016, 2017년에 대해 각각 영어, 수학
df = pd.DataFrame(np.random.randint(50, 100, 20).reshape(5, 4), index=['Kim', 'Park', 'Lee', 'Jung', 'Moon'],
                  columns=[[2016, 2016, 2017, 2017], ['영어', '수학', '영어', '수학']])
df.index.name='학생명'
df.columns.set_names(['연도', '과목'], inplace=True)
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    92  97   56  50
Park   73  95   57  97
Lee    54  70   53  87
Jung   62  94   81  70
Moon   69  62   98  71

In [74]:
# 기술통계, 요약통계
df.describe()

연도          2016                  2017           
과목            영어         수학         영어         수학
count   5.000000   5.000000   5.000000   5.000000
mean   70.000000  83.600000  69.000000  75.000000
std    14.265343  16.349312  19.710403  17.986106
min    54.000000  62.000000  53.000000  50.000000
25%    62.000000  70.000000  56.000000  70.000000
50%    69.000000  94.000000  57.000000  71.000000
75%    73.000000  95.000000  81.000000  87.000000
max    92.000000  97.000000  98.000000  97.000000

In [75]:
# 2017년도 성적만 분리해서 df2017로 저장(깊은복사)
df2017 = df[2017].copy()
df2017

과목,영어,수학
학생명,,
Kim,56,50
Park,57,97
Lee,53,87
Jung,81,70
Moon,98,71


In [76]:
# 1) count - NaN을 제외한 데이터의 개수(행 개수)
# 축 기준 설정 : axis를 통해 행 / 열 개수인지 지정 가능
# 행 개수 : axis = 0
df2017.count(axis=0)

과목
영어    5
수학    5
dtype: int64

In [77]:
# 열 개수 -> axis = 1
df2017.count(axis=1)

학생명
Kim     2
Park    2
Lee     2
Jung    2
Moon    2
dtype: int64

In [78]:
data

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [86]:
# value_count() : 변수형 데이터(문자열 등)에 대한 각 분류값의 개수
# 도시, 자치구 데이터 사용(data)
data.도시.value_counts()

서울    35
부산    15
Name: 도시, dtype: int64

In [87]:
# 2) sum() : 값의 총합을 계산
# 기본 축 방향 : 열별 총합
# == 과목별 총합
# axis = 0
df2017.sum()

과목
영어    345
수학    375
dtype: int64

In [88]:
# 학생별 점수 총합
df2017.sum(axis=1)

학생명
Kim     106
Park    154
Lee     140
Jung    151
Moon    169
dtype: int64

In [89]:
# 3) maen()
# 과목별 평균
# axis=0(디폴트)
df2017.mean()

과목
영어    69.0
수학    75.0
dtype: float64

In [101]:
df2017.mean(axis=1)

학생명
Kim     53.0
Park    77.0
Lee     70.0
Jung    75.5
Moon    84.5
dtype: float64

In [102]:
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    92  97   56  50
Park   73  95   57  97
Lee    54  70   53  87
Jung   62  94   81  70
Moon   69  62   98  71

In [110]:
# 문제 1 : df변수를 이용해서 2016, 2017 년도별, 과목별 평균성적 출력
df.mean()

연도    과목
2016  영어    70.0
      수학    83.6
2017  영어    69.0
      수학    75.0
dtype: float64

In [107]:
# 위의 년도별, 과목별 평균의 로우가 연도로, 컬럼이 과목이 되도록 수정해주세요.
df.mean().unstack()

과목,수학,영어
연도,,
2016,83.6,70.0
2017,75.0,69.0


In [113]:
# 문제 2 : df 변수를 이용해서 학생별로 전체 평균을 출력
df.mean(axis=1)

학생명
Kim     73.75
Park    80.50
Lee     66.00
Jung    76.75
Moon    75.00
dtype: float64